In [1]:
import csv
import json
from tqdm import tqdm
import pickle

In [2]:
file = open('../PKUBASE/pkubase-complete.txt','r',encoding='utf-8').readlines()

In [16]:
# this cell is just for test
for i in tqdm(range(len(file))):
    triplet = file[i].split('\t')
    
    if len(triplet)>2:
        entity1 = triplet[0]
        entity2 = triplet[2].rstrip(' .\n')
        relation = triplet[1]
    
    if i<30:
        print(triplet)
    else:
        break

#         if entity1[0]=='\"':
#             print(entity1, relation, entity2)
#             print(triplet)
#             entity1=entity1.strip('\"')
#         if entity2[0]=='\"':
#             print(entity1, relation, entity2)
#             print(triplet)
#             entity2=entity2.strip('\"')


        

  0%|          | 98407/66499746 [00:00<02:27, 450208.34it/s]

['<美国奥可斯（香港）国际控股集团>', '<类型>', '<文学作品> .\n']
['<美国奥可斯（香港）国际控股集团>', '<类型>', '<文化> .\n']
['<寻美中国>', '<类型>', '<品牌> .\n']
['<青春是我和你一杯酒的深>', '<类型>', '<文学作品> .\n']
['<青春是我和你一杯酒的深>', '<类型>', '<网络小说> .\n']
['<青春是我和你一杯酒的深>', '<类型>', '<小说作品> .\n']
['<青春是我和你一杯酒的深>', '<类型>', '<小说> .\n']
['<青春是我和你一杯酒的深>', '<类型>', '<娱乐作品> .\n']
['<青春是我和你一杯酒的深>', '<类型>', '<书籍> .\n']
['<青春是我和你一杯酒的深>', '<类型>', '<中国文学> .\n']
['<天使与猎人>', '<类型>', '<文学作品> .\n']
['<天使与猎人>', '<类型>', '<网络小说> .\n']
['<天使与猎人>', '<类型>', '<小说作品> .\n']
['<天使与猎人>', '<类型>', '<小说> .\n']
['<天使与猎人>', '<类型>', '<娱乐作品> .\n']
['<天使与猎人>', '<类型>', '<书籍> .\n']
['<天使与猎人>', '<类型>', '<中国文学> .\n']
['<天使与猎人>', '<类型>', '<玄幻小说> .\n']
['<霸道王妃>', '<类型>', '<文学作品> .\n']
['<霸道王妃>', '<类型>', '<网络小说> .\n']
['<霸道王妃>', '<类型>', '<小说作品> .\n']
['<霸道王妃>', '<类型>', '<小说> .\n']
['<霸道王妃>', '<类型>', '<书籍> .\n']
['<霸道王妃>', '<类型>', '<中国文学> .\n']
['<四库提要宋代总集研究>', '<类型>', '<书籍> .\n']
['<四库提要宋代总集研究>', '<类型>', '<历史书籍> .\n']
['<四库提要宋代总集研究>', '<类型>', '<出版物> .\n']
['<倾国怨伶新版>', '<类型>', '<娱乐作品> .\

 12%|█▏        | 8045010/66499746 [00:11<01:24, 693060.23it/s]


KeyboardInterrupt: 

In [5]:
def gen_node_file():

    file = open('../PKUBASE/pkubase-complete.txt','r',encoding='utf-8').readlines()
    print('数据库原始文件读取完成')
    csvf = open('node.csv','w',newline='',encoding = 'utf-8')

    w = csv.writer(csvf)
    w.writerow(("id:ID","name",":LABEL"))
    idx = 0
    entity_dic = dict()
    print(len(file))

    for i in tqdm(range(len(file))):
#         if '<类型>' in file[i]:
#             continue
#2019.8.30这次导入将实体的类型属性也导入数据库中
            
        triplet = file[i].split('\t')
        entity1 = triplet[0]
        if entity1 not in entity_dic:
            idx+=1
            entity_dic[entity1]=idx
            w.writerow((str(idx),entity1,'Entity'))
        
        if len(triplet)>2:# 部分数据由于没有实体和关系间没有加\t导致被识别为一个str
            entity2 = file[i].split('\t')[2].rstrip(' .\n')
            if entity2 not in entity_dic:
                idx+=1
                entity_dic[entity2] = idx
                w.writerow((str(idx),entity2,'Entity'))
        else:
            print(triplet)

    print(len(entity_dic))
    csvf.close()
    return entity_dic
# entity_dic = gen_node_file()
# print(len(entity_dic))

In [7]:
# 写入并保存relation.csv
csvf = open('relation.csv','w',newline='',encoding = 'utf-8')
w = csv.writer(csvf)
w.writerow((":START_ID",":END_ID",":TYPE","name"))

for i in tqdm(range(len(file))): 
#     if '<类型>' in file[i]:
#         continue
#2019.8.30更新
    triplet = file[i].split('\t')
    if len(triplet)>2:
        entity1 = triplet[0]
        entity2 = triplet[2].rstrip(' .\n')
        relation = triplet[1]

        w.writerow((str(entity_dic[entity1]),str(entity_dic[entity2]),'Relation',relation)) 


csvf.close()
print('finished.')

  0%|          | 50902/66499746 [00:00<04:23, 252475.93it/s]

25574425


100%|██████████| 66499746/66499746 [04:39<00:00, 237997.05it/s]


obtain property dictionary

In [8]:
atti_dic={}
# 属性dict, 属性是以双引号开始与结尾的字符串
# 例如：<南窑沟村> <中文名> "南窑沟村"
#       <埃琳娜·阿扎科娃> <出生日期> "1989-09-08"
#       <阿尔布林历代记> <其他名称> "Chronicles of Albian: The Magic Convention"

# 这里下标表示实体str的长度
for i in range(1,31):
    atti_dic[i]={}
for i in tqdm(range(len(file))):
    triplet = file[i].split('\t')
    if len(triplet)>2:
        entity1 = triplet[0]
        entity2 = triplet[2].rstrip(' .\n')
        try:
            if entity1[0]=='\"':
#                 print(entity1)
                entity1=entity1.strip('\"')
                atti_dic[len(entity1)][entity1]=''
            if entity2[0]=='\"':
                entity2=entity2.strip('\"')
                atti_dic[len(entity2)][entity2]=''
        except:
            continue
# print(atti_dic[30]) # 打印str长度为30的实体

100%|██████████| 66499746/66499746 [02:01<00:00, 545840.63it/s]


In [9]:
atti_dic[4]

{'霸道王妃': '',
 '1983': '',
 '听雨纪谈': '',
 '2011': '',
 '延迟策略': '',
 '1950': '',
 '企业保险': '',
 '整个冬季': '',
 '2003': '',
 '1927': '',
 '128亩': '',
 '香港珍珠': '',
 '丹武真仙': '',
 '南窑沟村': '',
 '魔心深黑': '',
 '珍璃尺蛾': '',
 '2000': '',
 '离休干部': '',
 '金庸无双': '',
 '15.0': '',
 '扶手软椅': '',
 '1961': '',
 '1993': '',
 '左右结构': '',
 '异界大陆': '',
 '公众作品': '',
 '1990': '',
 '膨化谷物': '',
 '清月无华': '',
 '1988': '',
 '1948': '',
 '狼疮因子': '',
 '无棣战役': '',
 '纸圣传奇': '',
 '2012': '',
 '中国五省': '',
 '1954': '',
 '背神经管': '',
 '繁花一尽': '',
 '纤细盗龙': '',
 '金丝厂组': '',
 '彩拓拍播': '',
 '春季为主': '',
 '1886': '',
 '1969': '',
 '香港时间': '',
 '齐藤雄大': '',
 '42英寸': '',
 '贝迪亚科': '',
 '洛浦街道': '',
 '平方公里': '',
 '2007': '',
 '20.0': '',
 '荷兰沙司': '',
 '1米78': '',
 '69公斤': '',
 '无爱之觞': '',
 '1932': '',
 '2001': '',
 '142米': '',
 '1903': '',
 '童话丛生': '',
 '奇迹之外': '',
 '湖南卫视': '',
 '节目嘉宾': '',
 '尤里安蒂': '',
 '53公斤': '',
 '1852': '',
 '1881': '',
 '1944': '',
 'QQ美食': '',
 '鬼才俊王': '',
 '神剑天狗': '',
 '粉碎德军': '',
 '正义无畏': '',
 '应用进程': '',
 '艾斯普顿': '',

In [10]:
atti_dic.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [11]:
pickle.dump(atti_dic,open('prop_dic.pkl','wb'))

obtain type dictionary

In [12]:
type_dic={} #保存实体的所有类型 key是实体，value是类型组成的list
for i in tqdm(range(len(file))):
    triplet = file[i].split('\t')
    if len(triplet)==3:
        if triplet[1]=='<类型>':
            entity = triplet[0]
            ent_type = triplet[2].rstrip(' .\n') # 关系是<类型>，entity2就是entity1的类型
            if entity in type_dic:
                type_dic[entity].append(ent_type)
            else:
                type_dic[entity]=[ent_type]

100%|██████████| 66499746/66499746 [02:40<00:00, 414632.90it/s] 


In [13]:
pickle.dump(type_dic,open('type_dic.pkl','wb'))

In [14]:
import pickle
type_dic=pickle.load(open('type_dic.pkl','rb'))
len(type_dic)

10216529

In [18]:
print(type_dic['<爱利亚的芝诺>'])
print(type_dic['<天使与猎人>'])

['<行业人物>', '<人物>']
['<文学作品>', '<网络小说>', '<小说作品>', '<小说>', '<娱乐作品>', '<书籍>', '<中国文学>', '<玄幻小说>']
